<img align="left" style="padding-right: 5px" valign="center" src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" width="28px"> 

# Intermediate pandas


*By Joseph Nelson, Jeff Hale and unknown other GA folks*

## Learning Objectives:

After this lesson, students will be able to:
- Use the `groupby` method to aggregate data by group
- Use `replace`, `value_counts`, `unique`, `nunique`, and `sample` on a DataFrame
- Combine DataFrames with `concat`
- Do joins with `merge`

#### Prior knowledge required:

- Python basics
- Pandas basics

Pandas Documentation on [Merging data](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

<div id="import"></div>
<h2>Import Pandas</h2>

In [5]:
import pandas as pd
import numpy as np
print(f'Pandas v{pd.__version__}\n Numpy v{np.__version__}')

Pandas v1.2.4
 Numpy v1.20.1



--- 
# Split-Apply-Combine with `groupby`

What if we want summary statistics _with respect to some categorical variable?_ 

For example, the price of a diamond probably varies widely between different diamond cuts. To tackle this problem, we'll use the **Split-Apply-Combine** technique.

* **Split**: Separate your data into different DataFrames, one for each category.
* **Apply**: On each split-up DataFrame, apply some function or transformation (for example, the mean).
* **Combine**: Take the results and combine the split-up DataFrames back into one aggregate DataFrame.

#### **Slack**: Import the Airbnb dataset

In [6]:
airbnb = pd.read_csv("./data/airbnb_listings.csv")

In [4]:
airbnb.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2265,Zen-East in the Heart of Austin (monthly rental),2466,Paddy,NaN,78702,30.27752,-97.71377,Entire home/apt,200,7,26,2021-07-02,0.38,3,51
1,5245,"Eco friendly, Colorful, Clean, Cozy monthly share",2466,Paddy,NaN,78702,30.27614,-97.71320,Private room,114,30,9,2017-02-24,0.22,3,0


#### Inspect

In [5]:
airbnb.describe()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.035000e+04,1.035000e+04,0.0,10350.000000,10350.000000,10350.000000,10350.000000,10350.000000,10350.000000,7944.000000,10350.000000,10350.000000
mean,2.836757e+07,9.792277e+07,NaN,78723.159420,30.279833,-97.752227,277.324831,7.714396,32.915652,2.485487,10.415652,116.768889
std,1.608979e+07,1.118991e+08,NaN,20.805698,0.059749,0.059600,546.320050,24.376424,69.867453,6.855260,27.583589,130.627737
min,2.265000e+03,2.300000e+01,NaN,78701.000000,30.078870,-98.056630,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,1.504138e+07,1.170741e+07,NaN,78704.000000,30.244343,-97.769207,90.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,2.888472e+07,4.744496e+07,NaN,78722.000000,30.269655,-97.744630,150.000000,2.000000,6.000000,0.900000,1.000000,64.000000
75%,4.351866e+07,1.498711e+08,NaN,78745.000000,30.302877,-97.720460,275.000000,3.000000,30.000000,2.510000,4.000000,233.000000
max,5.090360e+07,4.110306e+08,NaN,78759.000000,30.515090,-97.560620,16845.000000,1100.000000,917.000000,250.000000,158.000000,365.000000


In [6]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10350 entries, 0 to 10349
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              10350 non-null  int64  
 1   name                            10350 non-null  object 
 2   host_id                         10350 non-null  int64  
 3   host_name                       10334 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   10350 non-null  int64  
 6   latitude                        10350 non-null  float64
 7   longitude                       10350 non-null  float64
 8   room_type                       10350 non-null  object 
 9   price                           10350 non-null  int64  
 10  minimum_nights                  10350 non-null  int64  
 11  number_of_reviews               10350 non-null  int64  
 12  last_review                     

#### What is the mean price by room type?

In [7]:
airbnb['room_type'].unique()

array(['Entire home/apt', 'Private room', 'Hotel room', 'Shared room'],
      dtype=object)

In [8]:
airbnb['room_type'].value_counts()

Entire home/apt    8168
Private room       2050
Shared room         121
Hotel room           11
Name: room_type, dtype: int64

In [9]:
airbnb.groupby('room_type').mean()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
room_type,,,,,,,,,,,,
Entire home/apt,2.900129e+07,9.900012e+07,NaN,78721.379163,30.278627,-97.754818,318.310847,7.969148,36.388222,2.778648,11.768242,124.395813
Hotel room,2.891924e+07,1.212280e+08,NaN,78715.363636,30.288099,-97.787508,579.727273,2.000000,13.909091,0.653333,3.545455,116.454545
Private room,2.605988e+07,9.536797e+07,NaN,78729.594146,30.283663,-97.742918,124.733171,6.378049,20.702439,1.366372,3.902439,84.343415
Shared room,2.463590e+07,6.636223e+07,NaN,78735.024793,30.295572,-97.731860,68.338843,13.677686,7.148760,0.537609,30.082645,151.305785


In [10]:
airbnb.groupby('room_type')['price'].mean()

room_type
Entire home/apt    318.310847
Hotel room         579.727273
Private room       124.733171
Shared room         68.338843
Name: price, dtype: float64

In [11]:
airbnb.groupby('room_type')[['price' , 'number_of_reviews']].mean()

,price,number_of_reviews
room_type,,
Entire home/apt,318.310847,36.388222
Hotel room,579.727273,13.909091
Private room,124.733171,20.702439
Shared room,68.338843,7.148760


#### Sort the results

In [12]:
airbnb.groupby('room_type')['price'].mean().sort_values()

room_type
Shared room         68.338843
Private room       124.733171
Entire home/apt    318.310847
Hotel room         579.727273
Name: price, dtype: float64

In [13]:
airbnb.groupby('room_type')['price'].mean().sort_values(ascending = False)

room_type
Hotel room         579.727273
Entire home/apt    318.310847
Private room       124.733171
Shared room         68.338843
Name: price, dtype: float64

In [14]:
airbnb.groupby('room_type').mean().sort_values(by = 'price', ascending = False)

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
room_type,,,,,,,,,,,,
Hotel room,2.891924e+07,1.212280e+08,NaN,78715.363636,30.288099,-97.787508,579.727273,2.000000,13.909091,0.653333,3.545455,116.454545
Entire home/apt,2.900129e+07,9.900012e+07,NaN,78721.379163,30.278627,-97.754818,318.310847,7.969148,36.388222,2.778648,11.768242,124.395813
Private room,2.605988e+07,9.536797e+07,NaN,78729.594146,30.283663,-97.742918,124.733171,6.378049,20.702439,1.366372,3.902439,84.343415
Shared room,2.463590e+07,6.636223e+07,NaN,78735.024793,30.295572,-97.731860,68.338843,13.677686,7.148760,0.537609,30.082645,151.305785


#### Using the `.agg` method

In [15]:
airbnb.groupby('room_type').agg(np.mean)

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
room_type,,,,,,,,,,,,
Entire home/apt,2.900129e+07,9.900012e+07,NaN,78721.379163,30.278627,-97.754818,318.310847,7.969148,36.388222,2.778648,11.768242,124.395813
Hotel room,2.891924e+07,1.212280e+08,NaN,78715.363636,30.288099,-97.787508,579.727273,2.000000,13.909091,0.653333,3.545455,116.454545
Private room,2.605988e+07,9.536797e+07,NaN,78729.594146,30.283663,-97.742918,124.733171,6.378049,20.702439,1.366372,3.902439,84.343415
Shared room,2.463590e+07,6.636223e+07,NaN,78735.024793,30.295572,-97.731860,68.338843,13.677686,7.148760,0.537609,30.082645,151.305785


In [16]:
airbnb.groupby('room_type')[['price', 'number_of_reviews']].agg(np.mean)

,price,number_of_reviews
room_type,,
Entire home/apt,318.310847,36.388222
Hotel room,579.727273,13.909091
Private room,124.733171,20.702439
Shared room,68.338843,7.148760


In [17]:
airbnb.groupby('room_type')[['price', 'number_of_reviews']].agg([np.mean, np.median])

price        number_of_reviews       
                       mean median              mean median
room_type                                                  
Entire home/apt  318.310847    175         36.388222      7
Hotel room       579.727273    210         13.909091      1
Private room     124.733171     65         20.702439      3
Shared room       68.338843     28          7.148760      3

In [18]:
airbnb_aggs = airbnb.groupby('room_type')[['price', 'number_of_reviews']].agg([np.mean, np.median])

In [40]:
room_stats = airbnb.groupby('room_type').agg([np.mean, np.median])

In [41]:
room_stats.head(2)

id                 host_id            \
                         mean    median          mean    median   
room_type                                                         
Entire home/apt  2.900129e+07  30337209  9.900012e+07  46485292   
Hotel room       2.891924e+07  32171574  1.212280e+08  77021411   

                neighbourhood_group        neighbourhood            latitude  \
                               mean median          mean   median       mean   
room_type                                                                      
Entire home/apt                 NaN    NaN  78721.379163  78705.0  30.278627   
Hotel room                      NaN    NaN  78715.363636  78701.0  30.288099   

                           ... minimum_nights        number_of_reviews         \
                   median  ...           mean median              mean median   
room_type                  ...                                                  
Entire home/apt  30.26829  ...       7.969148      2         36.388222      7   
Hotel room       30.27191  ...       2.000000      2         13.909091      1   

                reviews_per_month        calculated_host_listings_count  \
                             mean median                           mean   
room_type                                                                 
Entire home/apt          2.778648  1.035                      11.768242   
Hotel room               0.653333  0.060                       3.545455   

                       availability_365         
                median             mean median  
room_type                                       
Entire home/apt      1       124.395813     77  
Hotel room           3       116.454545      0  

[2 rows x 24 columns]

In [42]:
room_stats['minimum_nights']['mean']

room_type
Entire home/apt     7.969148
Hotel room          2.000000
Private room        6.378049
Shared room        13.677686
Name: mean, dtype: float64

#### Now you have a multi-index column. In general, I suggest you avoid hierarchical indexes whenever possible - they can be tricky. How would you figure out how to flatten a multi-index?

In [19]:
df.columns = df.columns.get_level_values(0)
Link: https://stackoverflow.com/questions/14507794/pandas-how-to-flatten-a-hierarchical-index-in-columns

In [26]:
room_stats.columns = room_stats.columns.get_level_values(0)

In [27]:
room_stats.columns = room_stats.columns.map('_'.join)

In [69]:
room_stats

,id_mean,id_median,host_id_mean,host_id_median,neighbourhood_group_mean,neighbourhood_group_median,neighbourhood_mean,neighbourhood_median,latitude_mean,latitude_median,...,minimum_nights_mean,minimum_nights_median,number_of_reviews_mean,number_of_reviews_median,reviews_per_month_mean,reviews_per_month_median,calculated_host_listings_count_mean,calculated_host_listings_count_median,availability_365_mean,availability_365_median
room_type,,,,,,,,,,,,,,,,,,,,,
Entire home/apt,2.900129e+07,30337209,9.900012e+07,46485292,NaN,NaN,78721.379163,78705.0,30.278627,30.26829,...,7.969148,2,36.388222,7,2.778648,1.035,11.768242,1,124.395813,77
Hotel room,2.891924e+07,32171574,1.212280e+08,77021411,NaN,NaN,78715.363636,78701.0,30.288099,30.27191,...,2.000000,2,13.909091,1,0.653333,0.060,3.545455,3,116.454545,0
Private room,2.605988e+07,23715203,9.536797e+07,51358009,NaN,NaN,78729.594146,78733.5,30.283663,30.27330,...,6.378049,1,20.702439,3,1.366372,0.380,3.902439,1,84.343415,0
Shared room,2.463590e+07,21881807,6.636223e+07,20523885,NaN,NaN,78735.024793,78746.0,30.295572,30.30390,...,13.677686,2,7.148760,3,0.537609,0.140,30.082645,3,151.305785,90


In [ ]:
Link: https://stackoverflow.com/questions/50571793/concise-way-of-flattening-multiindex-columns

In [29]:
room_stats.columns = ['_'.join(col) for col in room_stats.columns.values]

In [35]:
room_stats

id                 host_id            \
                         mean    median          mean    median   
room_type                                                         
Entire home/apt  2.900129e+07  30337209  9.900012e+07  46485292   
Hotel room       2.891924e+07  32171574  1.212280e+08  77021411   
Private room     2.605988e+07  23715203  9.536797e+07  51358009   
Shared room      2.463590e+07  21881807  6.636223e+07  20523885   

                neighbourhood_group        neighbourhood            latitude  \
                               mean median          mean   median       mean   
room_type                                                                      
Entire home/apt                 NaN    NaN  78721.379163  78705.0  30.278627   
Hotel room                      NaN    NaN  78715.363636  78701.0  30.288099   
Private room                    NaN    NaN  78729.594146  78733.5  30.283663   
Shared room                     NaN    NaN  78735.024793  78746.0  30.295572   

                           ... minimum_nights        number_of_reviews         \
                   median  ...           mean median              mean median   
room_type                  ...                                                  
Entire home/apt  30.26829  ...       7.969148      2         36.388222      7   
Hotel room       30.27191  ...       2.000000      2         13.909091      1   
Private room     30.27330  ...       6.378049      1         20.702439      3   
Shared room      30.30390  ...      13.677686      2          7.148760      3   

                reviews_per_month        calculated_host_listings_count  \
                             mean median                           mean   
room_type                                                                 
Entire home/apt          2.778648  1.035                      11.768242   
Hotel room               0.653333  0.060                       3.545455   
Private room             1.366372  0.380                       3.902439   
Shared room              0.537609  0.140                      30.082645   

                       availability_365         
                median             mean median  
room_type                                       
Entire home/apt      1       124.395813     77  
Hotel room           3       116.454545      0  
Private room         1        84.343415      0  
Shared room          3       151.305785     90  

[4 rows x 24 columns]

In [36]:
room_stats.columns = ["_".join(a) for a in room_stats.columns.to_flat_index()]

In [37]:
room_stats

,id_mean,id_median,host_id_mean,host_id_median,neighbourhood_group_mean,neighbourhood_group_median,neighbourhood_mean,neighbourhood_median,latitude_mean,latitude_median,...,minimum_nights_mean,minimum_nights_median,number_of_reviews_mean,number_of_reviews_median,reviews_per_month_mean,reviews_per_month_median,calculated_host_listings_count_mean,calculated_host_listings_count_median,availability_365_mean,availability_365_median
room_type,,,,,,,,,,,,,,,,,,,,,
Entire home/apt,2.900129e+07,30337209,9.900012e+07,46485292,NaN,NaN,78721.379163,78705.0,30.278627,30.26829,...,7.969148,2,36.388222,7,2.778648,1.035,11.768242,1,124.395813,77
Hotel room,2.891924e+07,32171574,1.212280e+08,77021411,NaN,NaN,78715.363636,78701.0,30.288099,30.27191,...,2.000000,2,13.909091,1,0.653333,0.060,3.545455,3,116.454545,0
Private room,2.605988e+07,23715203,9.536797e+07,51358009,NaN,NaN,78729.594146,78733.5,30.283663,30.27330,...,6.378049,1,20.702439,3,1.366372,0.380,3.902439,1,84.343415,0
Shared room,2.463590e+07,21881807,6.636223e+07,20523885,NaN,NaN,78735.024793,78746.0,30.295572,30.30390,...,13.677686,2,7.148760,3,0.537609,0.140,30.082645,3,151.305785,90


In [43]:
room_stats.columns = ["_".join(a) for a in room_stats.columns]

In [44]:
room_stats

,id_mean,id_median,host_id_mean,host_id_median,neighbourhood_group_mean,neighbourhood_group_median,neighbourhood_mean,neighbourhood_median,latitude_mean,latitude_median,...,minimum_nights_mean,minimum_nights_median,number_of_reviews_mean,number_of_reviews_median,reviews_per_month_mean,reviews_per_month_median,calculated_host_listings_count_mean,calculated_host_listings_count_median,availability_365_mean,availability_365_median
room_type,,,,,,,,,,,,,,,,,,,,,
Entire home/apt,2.900129e+07,30337209,9.900012e+07,46485292,NaN,NaN,78721.379163,78705.0,30.278627,30.26829,...,7.969148,2,36.388222,7,2.778648,1.035,11.768242,1,124.395813,77
Hotel room,2.891924e+07,32171574,1.212280e+08,77021411,NaN,NaN,78715.363636,78701.0,30.288099,30.27191,...,2.000000,2,13.909091,1,0.653333,0.060,3.545455,3,116.454545,0
Private room,2.605988e+07,23715203,9.536797e+07,51358009,NaN,NaN,78729.594146,78733.5,30.283663,30.27330,...,6.378049,1,20.702439,3,1.366372,0.380,3.902439,1,84.343415,0
Shared room,2.463590e+07,21881807,6.636223e+07,20523885,NaN,NaN,78735.024793,78746.0,30.295572,30.30390,...,13.677686,2,7.148760,3,0.537609,0.140,30.082645,3,151.305785,90


#### How do you groupby `room_type` and `neighborhood`?

In [45]:
airbnb.groupby(['room_type', 'neighbourhood']).median()

id      host_id  neighbourhood_group  \
room_type       neighbourhood                                                 
Entire home/apt 78701          38070578.0   74292291.0                  NaN   
                78702          28962698.5   26440310.5                  NaN   
                78703          22172050.0   28648585.0                  NaN   
                78704          23319078.0   28767982.0                  NaN   
                78705          29240059.0   88566861.0                  NaN   
...                                   ...          ...                  ...   
Shared room     78752          32839654.0   20484642.0                  NaN   
                78753          37048493.0  125839353.0                  NaN   
                78756          27547539.0    8167447.0                  NaN   
                78757          33476154.5  104826449.0                  NaN   
                78758          37173968.5  200360713.0                  NaN   

                                latitude  longitude  price  minimum_nights  \
room_type       neighbourhood                                                
Entire home/apt 78701          30.271075 -97.747035  240.0             2.0   
                78702          30.262225 -97.720410  198.0             2.0   
                78703          30.280970 -97.760010  210.0             2.0   
                78704          30.246710 -97.760810  182.0             2.0   
                78705          30.291005 -97.744305  150.0             2.0   
...                                  ...        ...    ...             ...   
Shared room     78752          30.329450 -97.699750  100.0             1.0   
                78753          30.390700 -97.647480   15.0             1.0   
                78756          30.319970 -97.730960   17.0            30.0   
                78757          30.355650 -97.728485   35.0            15.5   
                78758          30.371275 -97.712495  164.0             2.0   

                               number_of_reviews  reviews_per_month  \
room_type       neighbourhood                                         
Entire home/apt 78701                        2.0              0.590   
                78702                       15.0              1.500   
                78703                        5.0              0.840   
                78704                        8.0              0.910   
                78705                        6.0              1.110   
...                                          ...                ...   
Shared room     78752                        0.0                NaN   
                78753                        5.0              0.240   
                78756                        4.0              0.135   
                78757                        0.0                NaN   
                78758                        0.0                NaN   

                               calculated_host_listings_count  \
room_type       neighbourhood                                   
Entire home/apt 78701                                     4.5   
                78702                                     1.0   
                78703                                     1.0   
                78704                                     1.0   
                78705                                     1.0   
...                                                       ...   
Shared room     78752                                     1.0   
                78753                                     1.0   
                78756                                    76.0   
                78757                                     1.0   
                78758                                     1.0   

                               availability_365  
room_type       neighbourhood                    
Entire home/apt 78701                      48.0  
                78702                      75.0  
                78703     

In [46]:
airbnb.groupby(['room_type', 'neighbourhood'])['price'].median()

room_type        neighbourhood
Entire home/apt  78701            240.0
                 78702            198.0
                 78703            210.0
                 78704            182.0
                 78705            150.0
                                  ...  
Shared room      78752            100.0
                 78753             15.0
                 78756             17.0
                 78757             35.0
                 78758            164.0
Name: price, Length: 120, dtype: float64

___
## Exercises

#### Sort the airbnb DataFrame by most expensive to least expensive listing

In [56]:
airbnb.sort_values(by = 'price', ascending = False)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
8298,46081394,ABOVE PREMIERE I LAKEFRONT I INDOOR/OUTDOOR PO...,210733801,Above Vacation,NaN,78733,30.343100,-97.878650,Entire home/apt,16845,1,0,NaN,NaN,41,360
6344,36868625,Sapphire on Lake Austin,210733801,Above Vacation,NaN,78733,30.336170,-97.847720,Entire home/apt,14739,1,0,NaN,NaN,41,355
7436,42351709,WanderJaunt | Carolyn | 2BR | Hyde Park,104309976,WanderJaunt,NaN,78751,30.304020,-97.725380,Entire home/apt,10000,1,20,2021-06-20,1.22,142,0
4151,23003072,Amazing Luxury in Downtown Austin perfect for ...,384732,Andres,NaN,78701,30.268090,-97.749710,Entire home/apt,10000,500,1,2018-03-14,0.02,1,180
920,4538217,The Cathédrale,22773046,Eli,NaN,78723,30.317060,-97.683100,Entire home/apt,10000,30,3,2015-05-23,0.04,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3417,19840835,Eco Friendly Tiny Apartment,3440175,James,NaN,78702,30.254380,-97.703060,Private room,5,1,1,2017-07-16,0.02,8,0
3577,20674436,Hurricane Harvey Refuge ONLY,3594051,Michael,NaN,78704,30.239280,-97.742980,Entire home/apt,1,2,2,2017-09-10,0.04,1,0
3593,20708014,small room for Harvey evacuees,22987908,Elizabeth,NaN,78757,30.355100,-97.738090,Private room,1,1,0,NaN,NaN,1,0
8254,45895289,Hotel Saint Cecilia,260188286,Hotel Saint Cecilia,NaN,78704,30.251623,-97.746515,Hotel room,0,1,0,NaN,NaN,1,0


#### Show the max listing price for each neighbourhood

In [61]:
airbnb.groupby('neighbourhood')['price'].max().sort_values(ascending = False)

neighbourhood
78733    16845
78701    10000
78723    10000
78751    10000
78702    10000
78750     9998
78703     9718
78746     6948
78704     6316
78732     6159
78730     5474
78749     5021
78734     5000
78741     4999
78738     3696
78737     3073
78731     3000
78756     3000
78735     3000
78745     2500
78705     2450
78722     2000
78736     1880
78752     1764
78721     1665
78728     1499
78759     1250
78725     1200
78748     1157
78739     1099
78724     1050
78742     1000
78757     1000
78754     1000
78729     1000
78744      950
78753      900
78726      825
78717      700
78747      607
78727      574
78758      557
78712      500
78719      171
Name: price, dtype: int64

In [7]:
airbnb.groupby(['neighbourhood']).agg([np.max])

,id,name,host_id,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
,amax,amax,amax,amax,amax,amax,amax,amax,amax,amax,amax,amax,amax
neighbourhood,,,,,,,,,,,,,
78701,50790623,🏙 Modern Hip 1 BR High Rise Condo Pool Gameroom,410019817,NaN,30.28322,-97.734110,Shared room,10000,500,819,150.00,142,365
78702,50896809,🥂Rainey Street—6 min walk ♫ 6th Street—3 min d...,410169720,NaN,30.28209,-97.692880,Shared room,10000,180,763,63.12,158,365
78703,50896316,"★The ATX Oasis★ FUN, 3 story house w/ pool & ...",404924711,NaN,30.31267,-97.750110,Shared room,9718,999,509,30.99,142,365
78704,50901766,🏯 LIMITED DISCOUNT Cozy Oriental Extended Stay...,411030626,NaN,30.26606,-97.735560,Shared room,6316,365,903,108.00,158,365
78705,50903601,★Only Airbnb in Austin w/ 5 desks★ WFH Ready!,383824821,NaN,30.30300,-97.721100,Shared room,2450,365,632,59.40,142,365
78712,10764579,Zat,27903197,NaN,30.28664,-97.733980,Private room,500,1,0,NaN,1,0
78717,50815145,true peace,400522896,NaN,30.51509,-97.722070,Private room,700,183,240,8.04,33,365
78719,43874171,The Sea Shell vintage tiny home,254119744,NaN,30.17627,-97.642740,Private room,171,3,113,2.64,33,319


#### Display the count of listings in each neighbourhood zip code

In [10]:
airbnb.groupby('neighbourhood')['calculated_host_listings_count'].count().sort_values(ascending = False)

neighbourhood
78704    1818
78702    1316
78701     733
78741     674
78745     511
78703     508
78705     495
78751     460
78723     330
78746     248
78758     244
78734     223
78721     213
78757     203
78744     182
78722     177
78752     146
78748     143
78756     141
78731     123
78753     113
78754     109
78737     107
78759     102
78749      99
78729      98
78727      87
78735      83
78724      78
78733      67
78732      63
78728      62
78717      58
78747      54
78736      52
78738      51
78725      44
78750      40
78730      38
78739      29
78726      12
78742       8
78719       7
78712       1
Name: calculated_host_listings_count, dtype: int64

#### Rename the _neighbourhood_ column to _zip_code_

In [67]:
airbnb.rename(columns = {'neighbourhood' : 'zip code'}, inplace = True)

In [68]:
airbnb.head(2)

,id,name,host_id,host_name,neighbourhood_group,zip code,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2265,Zen-East in the Heart of Austin (monthly rental),2466,Paddy,NaN,78702,30.27752,-97.71377,Entire home/apt,200,7,26,2021-07-02,0.38,3,51
1,5245,"Eco friendly, Colorful, Clean, Cozy monthly share",2466,Paddy,NaN,78702,30.27614,-97.71320,Private room,114,30,9,2017-02-24,0.22,3,0


#### How many rows have a listing price < $500?

In [8]:
airbnb[airbnb['price'] < 500]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2265,Zen-East in the Heart of Austin (monthly rental),2466,Paddy,NaN,78702,30.277520,-97.713770,Entire home/apt,200,7,26,2021-07-02,0.38,3,51
1,5245,"Eco friendly, Colorful, Clean, Cozy monthly share",2466,Paddy,NaN,78702,30.276140,-97.713200,Private room,114,30,9,2017-02-24,0.22,3,0
2,5456,"Walk to 6th, Rainey St and Convention Ctr",8028,Sylvia,NaN,78702,30.260570,-97.734410,Entire home/apt,126,2,560,2021-06-21,27.27,1,329
3,5769,NW Austin Room,8186,Elizabeth,NaN,78729,30.456970,-97.784220,Private room,39,1,264,2021-07-03,6.42,1,30
4,6413,Gem of a Studio near Downtown,13879,Todd,NaN,78704,30.248850,-97.735870,Entire home/apt,104,30,117,2021-04-02,1.31,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10345,50899252,Stay in a place of your own | 1BR in Austin,359036978,Shelby,NaN,78745,30.181934,-97.777781,Entire home/apt,117,30,0,NaN,NaN,158,365
10346,50899636,Brilliant apartment home | 1BR in Austin,359036978,Shelby,NaN,78729,30.447745,-97.783717,Entire home/apt,112,30,0,NaN,NaN,158,365
10347,50899637,Brilliant apartment home | 1BR in Austin,359036978,Shelby,NaN,78729,30.448854,-97.784034,Entire home/apt,119,30,0,NaN,NaN,158,365
10348,50901766,Mid-Century on a corner lot with fenced yard,80406,Dani,NaN,78704,30.241941,-97.742844,Entire home/apt,199,7,0,NaN,NaN,12,63


In [9]:
airbnb.loc[airbnb['price'] < 500].count()

id                                9114
name                              9114
host_id                           9114
host_name                         9099
neighbourhood_group                  0
neighbourhood                     9114
latitude                          9114
longitude                         9114
room_type                         9114
price                             9114
minimum_nights                    9114
number_of_reviews                 9114
last_review                       7179
reviews_per_month                 7179
calculated_host_listings_count    9114
availability_365                  9114
dtype: int64

In [11]:
airbnb.loc[airbnb['price'] < 500]['price'].count()

9114

In [12]:
airbnb.loc[airbnb['price'] < 500].shape[0]

9114

#### Create a new column called `listing_type`. For each row that has a room type of _Hotel room_, fill this new column with the value _hotel_. If the room type is **not** a hotel room, fill this new column with the value _home_. 
##### Hint: We covered this earlier in the intro pandas section. You're welcome to use the internet as well to help!

___

<div id="conapp"></div>
<h2>Combine DataFrames with pd.concat()</h2>

## Concatenate

*Concatenate* means to sticks DataFrames together -  either on top of each other, or next to each other.

First, let's create two DataFrames for hypothetical soccer data, both with standings data for different teams.

In [13]:
standings1 = pd.DataFrame(
    [['usa', 3, 4, 1], 
    ['canada', 7, 2, 0], 
    ['spain', 4, 5, 2], 
    ['nigeria', 4, 3, 3], 
    ['argentina', 5, 2, 1]], 
    columns=['team', 'wins', 'losses', 'ties']
)
standings1.head()

,team,wins,losses,ties
0,usa,3,4,1
1,canada,7,2,0
2,spain,4,5,2
3,nigeria,4,3,3
4,argentina,5,2,1


In [14]:
standings2 = pd.DataFrame(
    [['jamaica', 13, 14, 11], 
    ['italy', 17, 12, 10], 
    ['paraquay', 14, 15, 12], 
    ['burkina faso', 14, 13, 13], 
    ['japan', 15, 2, 11]], 
    columns=['team', 'wins', 'losses', 'ties']
)
standings2.head()

,team,wins,losses,ties
0,jamaica,13,14,11
1,italy,17,12,10
2,paraquay,14,15,12
3,burkina faso,14,13,13
4,japan,15,2,11


#### Stick the dataframes on top of each other using `concat`. 

In [15]:
pd.concat([standings1, standings2])

,team,wins,losses,ties
0,usa,3,4,1
1,canada,7,2,0
2,spain,4,5,2
3,nigeria,4,3,3
4,argentina,5,2,1
0,jamaica,13,14,11
1,italy,17,12,10
2,paraquay,14,15,12
3,burkina faso,14,13,13
4,japan,15,2,11


#### Check out the index. How can we fix that?

In [16]:
pd.concat([standings1, standings2], ignore_index = True)

,team,wins,losses,ties
0,usa,3,4,1
1,canada,7,2,0
2,spain,4,5,2
3,nigeria,4,3,3
4,argentina,5,2,1
5,jamaica,13,14,11
6,italy,17,12,10
7,paraquay,14,15,12
8,burkina faso,14,13,13
9,japan,15,2,11


In [19]:
pd.concat([standings1, standings2]).reset_index(drop = True)

,team,wins,losses,ties
0,usa,3,4,1
1,canada,7,2,0
2,spain,4,5,2
3,nigeria,4,3,3
4,argentina,5,2,1
5,jamaica,13,14,11
6,italy,17,12,10
7,paraquay,14,15,12
8,burkina faso,14,13,13
9,japan,15,2,11


In [20]:
standings3 = pd.concat([standings1, standings2]).reset_index(drop = True)

In [21]:
standings3

,team,wins,losses,ties
0,usa,3,4,1
1,canada,7,2,0
2,spain,4,5,2
3,nigeria,4,3,3
4,argentina,5,2,1
5,jamaica,13,14,11
6,italy,17,12,10
7,paraquay,14,15,12
8,burkina faso,14,13,13
9,japan,15,2,11


#### Combine 3 DataFrames

In [22]:
pd.concat([standings1, standings2, standings3], ignore_index = True)

,team,wins,losses,ties
0,usa,3,4,1
1,canada,7,2,0
2,spain,4,5,2
3,nigeria,4,3,3
4,argentina,5,2,1
5,jamaica,13,14,11
6,italy,17,12,10
7,paraquay,14,15,12
8,burkina faso,14,13,13
9,japan,15,2,11


#### Side-by-side
Stick the DataFrames next to each other.

In [26]:
pd.concat([standings1, standings2], axis = 1)

,team,wins,losses,ties,team,wins,losses,ties
0,usa,3,4,1,jamaica,13,14,11
1,canada,7,2,0,italy,17,12,10
2,spain,4,5,2,paraquay,14,15,12
3,nigeria,4,3,3,burkina faso,14,13,13
4,argentina,5,2,1,japan,15,2,11


___
## Merging

### Merge DataFrames with `pd.merge()`

<div id="merge"></div>

### Inner join:

Only rows that match on the specified columns in both tables are kept.

### Left join:
All rows are kept from the left table. Only rows that match on the specified columns in the right table are kept. There may be NaNs.

### Outer join: 
All rows are kept. There may be NaNs. 


## In all cases, all the columns stick around

![joins venn diagrams](./images/joins.png) [Image source](https://www.howtoautomate.in.th/wp-content/uploads/2017/09/2017-09-23-12_38_26-SQL-Join-Types-%E2%80%94-SQL-Joins-Explained.png)


`merge` can effectively do everything that `join` can do.

Note that `merge` is <i>both</i> a DataFrame method as well as a top level pandas function. Below, we'll be using the pandas function, `pd.merge()`.

#### Let's make another DataFrame

Let's add goals information for some of those teams above

In [31]:
goals = pd.DataFrame(
    [['usa', 10, 14,], 
    ['canada', 15, 3], 
    ['italy', 8, 5], 
    ['japan', 14, 13], 
    ['argentina', 19, 12]], 
    columns=['team', 'goals_for', 'goals_against']
)
goals

,team,goals_for,goals_against
0,usa,10,14
1,canada,15,3
2,italy,8,5
3,japan,14,13
4,argentina,19,12


In [28]:
standings1

,team,wins,losses,ties
0,usa,3,4,1
1,canada,7,2,0
2,spain,4,5,2
3,nigeria,4,3,3
4,argentina,5,2,1


In [29]:
standings2

,team,wins,losses,ties
0,jamaica,13,14,11
1,italy,17,12,10
2,paraquay,14,15,12
3,burkina faso,14,13,13
4,japan,15,2,11


### The default merge type is `inner`. ☝️

In [32]:
pd.merge(goals, standings1)

,team,goals_for,goals_against,wins,losses,ties
0,usa,10,14,3,4,1
1,canada,15,3,7,2,0
2,argentina,19,12,5,2,1


In [33]:
pd.merge(goals, standings1, how = 'left')

,team,goals_for,goals_against,wins,losses,ties
0,usa,10,14,3.0,4.0,1.0
1,canada,15,3,7.0,2.0,0.0
2,italy,8,5,NaN,NaN,NaN
3,japan,14,13,NaN,NaN,NaN
4,argentina,19,12,5.0,2.0,1.0


In [35]:
pd.merge(left = goals, right = standings1, how = 'right')

,team,goals_for,goals_against,wins,losses,ties
0,usa,10.0,14.0,3,4,1
1,canada,15.0,3.0,7,2,0
2,spain,NaN,NaN,4,5,2
3,nigeria,NaN,NaN,4,3,3
4,argentina,19.0,12.0,5,2,1


In [36]:
pd.merge(left = goals, right = standings1, how = 'outer')

,team,goals_for,goals_against,wins,losses,ties
0,usa,10.0,14.0,3.0,4.0,1.0
1,canada,15.0,3.0,7.0,2.0,0.0
2,italy,8.0,5.0,NaN,NaN,NaN
3,japan,14.0,13.0,NaN,NaN,NaN
4,argentina,19.0,12.0,5.0,2.0,1.0
5,spain,NaN,NaN,4.0,5.0,2.0
6,nigeria,NaN,NaN,4.0,3.0,3.0


In [37]:
pd.merge(goals, standings1, how = 'left', on = 'team')

,team,goals_for,goals_against,wins,losses,ties
0,usa,10,14,3.0,4.0,1.0
1,canada,15,3,7.0,2.0,0.0
2,italy,8,5,NaN,NaN,NaN
3,japan,14,13,NaN,NaN,NaN
4,argentina,19,12,5.0,2.0,1.0


In [38]:
goals.merge(standings2, how = 'inner', on = 'team')

,team,goals_for,goals_against,wins,losses,ties
0,italy,8,5,17,12,10
1,japan,14,13,15,2,11


In [39]:
goals.rename(columns = {'team' : 'country'}, inplace = True)

In [40]:
goals

,country,goals_for,goals_against
0,usa,10,14
1,canada,15,3
2,italy,8,5
3,japan,14,13
4,argentina,19,12


In [41]:
pd.merge(goals, standings1, how = 'left')

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [42]:
pd.merge(goals, standings1, how = 'left', left_on = 'country', right_on = 'team')

,country,goals_for,goals_against,team,wins,losses,ties
0,usa,10,14,usa,3.0,4.0,1.0
1,canada,15,3,canada,7.0,2.0,0.0
2,italy,8,5,NaN,NaN,NaN,NaN
3,japan,14,13,NaN,NaN,NaN,NaN
4,argentina,19,12,argentina,5.0,2.0,1.0


In [43]:
pd.merge(goals, standings1, how = 'left', left_on = 'country', right_on = 'team').drop('team', axis = 1)

,country,goals_for,goals_against,wins,losses,ties
0,usa,10,14,3.0,4.0,1.0
1,canada,15,3,7.0,2.0,0.0
2,italy,8,5,NaN,NaN,NaN
3,japan,14,13,NaN,NaN,NaN
4,argentina,19,12,5.0,2.0,1.0


In [44]:
standings2

,team,wins,losses,ties
0,jamaica,13,14,11
1,italy,17,12,10
2,paraquay,14,15,12
3,burkina faso,14,13,13
4,japan,15,2,11


In [45]:
standings2['goals_for'] = [10, 8, 19, 20, 15]

In [46]:
standings2

,team,wins,losses,ties,goals_for
0,jamaica,13,14,11,10
1,italy,17,12,10,8
2,paraquay,14,15,12,19
3,burkina faso,14,13,13,20
4,japan,15,2,11,15


In [47]:
goals

,country,goals_for,goals_against
0,usa,10,14
1,canada,15,3
2,italy,8,5
3,japan,14,13
4,argentina,19,12


In [48]:
pd.merge(goals, standings2, how = 'inner', left_on = 'country', right_on = 'team')

,country,goals_for_x,goals_against,team,wins,losses,ties,goals_for_y
0,italy,8,5,italy,17,12,10,8
1,japan,14,13,japan,15,2,11,15


In [49]:
pd.merge(goals, standings2, how = 'inner', left_on = 'country', right_on = 'team', suffixes = ['', '_standings'])

,country,goals_for,goals_against,team,wins,losses,ties,goals_for_standings
0,italy,8,5,italy,17,12,10,8
1,japan,14,13,japan,15,2,11,15


<div id="exercise"></div>
<h2>Table Joins</h2>
<p align="right">
<img src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">
</p>

In [51]:
p = pd.read_csv('./data/Production.Product.csv', sep='\t')
ps = pd.read_csv('./data/Production.ProductSubcategory.csv', sep='\t')
soh = pd.read_csv('./data/Sales.SalesOrderHeader.csv', sep='\t', nrows=1000)
sod = pd.read_csv('./data/Sales.SalesOrderDetail.csv', sep='\t', nrows=1000)

### Change max number of columns you can see 

In [52]:
p.head(2)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000


In [57]:
pd.options.display.max_columns = None

In [58]:
p.head(2)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000


In [60]:
pd.options.display.max_rows = 150

In [61]:
p.head(2)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000


Check the DataFrame after changing _max_columns_

<div id="p_exercise"></div>
<h3>Table Joins Exercise</h3>

Here are the data dictionaries we'll be using for the following exercise:

<ul>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html">Production.Product</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubCategory.html">Production.ProductSubcategory</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderHeader.html">Sales.SalesOrderHeader</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderDetail.html">Sales.SalesOrderDetail</a></li>
</ul>

## Let's use an AdventureWorks Cycles Dataset
<img align="right" src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">

Here's the Production.Product table [data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html), which is a description of the fields (columns) in the table (the .csv file we will import below):<br>
- **ProductID** - Primary key for Product records.
- **Name** - Name of the product.
- **ProductNumber** - Unique product identification number.
- **MakeFlag** - 0 = Product is purchased, 1 = Product is manufactured in-house.
- **FinishedGoodsFlag** - 0 = Product is not a salable item. 1 = Product is salable.
- **Color** - Product color.
- **SafetyStockLevel** - Minimum inventory quantity.
- **ReorderPoint** - Inventory level that triggers a purchase order or work order.
- **StandardCost** - Standard cost of the product.
- **ListPrice** - Selling price.
- **Size** - Product size.
- **SizeUnitMeasureCode** - Unit of measure for the Size column.
- **WeightUnitMeasureCode** - Unit of measure for the Weight column.
- **DaysToManufacture** - Number of days required to manufacture the product.
- **ProductLine** - R = Road, M = Mountain, T = Touring, S = Standard
- **Class** - H = High, M = Medium, L = Low
- **Style** - W = Womens, M = Mens, U = Universal
- **ProductSubcategoryID** - Product is a member of this product subcategory. Foreign key to ProductSubCategory.ProductSubCategoryID.
- **ProductModelID** - Product is a member of this product model. Foreign key to ProductModel.ProductModelID.
- **SellStartDate** - Date the product was available for sale.
- **SellEndDate** - Date the product was no longer available for sale.
- **DiscontinuedDate** - Date the product was discontinued.
- **rowguid** - ROWGUIDCOL number uniquely identifying the record. Used to support a merge replication sample.
- **ModifiedDate** - Date and time the record was last updated.


<div id="ex_pp"></div>
<h4>#1 Join Tables</h4>

<ul>
    <li>Using the <code>Production.Product.ProductID</code> and <code>Sales.SalesOrderDetail.ProductID</code> keys, join only the rows that match in both the <code>Production.Product</code> and <code>Sales.SalesOrderDetail</code> tables.</li>
</ul>

In [63]:
p.head(2)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000


In [64]:
sod.head(2)

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,{B207C96D-D9E6-402B-8470-2CC176C42283},2011-05-31 00:00:00
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,{7ABB600D-1E77-41BE-9FE5-B9142CFC08FA},2011-05-31 00:00:00


In [68]:
pd.merge(p, sod, on = 'ProductID').head(3)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid_x,ModifiedDate_x,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid_y,ModifiedDate_y
0,707,"Sport-100 Helmet, Red",HL-U509-R,0,1,Red,4,3,13.0863,34.99,NaN,NaN,NaN,NaN,0,S,NaN,NaN,31.0,33.0,2011-05-31 00:00:00,NaN,NaN,{2E1EF41A-C08A-4FF6-8ADA-BDE58B64A712},2014-02-08 10:01:36.827000000,43665,63,19F0-4638-8E,1,1,20.1865,0.0,20.1865,{E4A00FC2-6EDA-40D7-AFB4-B6B4C5B3EDEE},2011-05-31 00:00:00
1,707,"Sport-100 Helmet, Red",HL-U509-R,0,1,Red,4,3,13.0863,34.99,NaN,NaN,NaN,NaN,0,S,NaN,NaN,31.0,33.0,2011-05-31 00:00:00,NaN,NaN,{2E1EF41A-C08A-4FF6-8ADA-BDE58B64A712},2014-02-08 10:01:36.827000000,43668,87,365D-4C9A-BE,2,1,20.1865,0.0,40.3730,{C0517F7A-8994-470A-BF62-D7D653972F2A},2011-05-31 00:00:00
2,707,"Sport-100 Helmet, Red",HL-U509-R,0,1,Red,4,3,13.0863,34.99,NaN,NaN,NaN,NaN,0,S,NaN,NaN,31.0,33.0,2011-05-31 00:00:00,NaN,NaN,{2E1EF41A-C08A-4FF6-8ADA-BDE58B64A712},2014-02-08 10:01:36.827000000,43673,139,260F-4DCF-A1,4,1,20.1865,0.0,80.7460,{AC5123F3-3719-4802-9A72-D9A90C721E60},2011-05-31 00:00:00


In [74]:
len(pd.merge(p, sod, on = 'ProductID'))

1000

<div id="ex_soh_sod"></div>
<h4>#2 Join Sales Order Header and Sales Order Detail Tables</h4>

<ul>
    <li>Join the <code>Sales.SalesOrderHeader</code> and <code>Sales.SalesOrderDetail</code> tables with an inner join</li>
    <li>Don't forget to use your data dictionary to find the columns to merge on!</li>
</ul>
    <li>**Advanced** : Can you think of a Pythonic way to find columns that are common to both dataframes?</li>
</ul>

In [69]:
soh.head(2)

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00
1,43660,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,NaN,{738DC42D-D03B-48A1-9822-F95A67EA7389},2011-06-07 00:00:00


In [70]:
sod.head(2)

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,{B207C96D-D9E6-402B-8470-2CC176C42283},2011-05-31 00:00:00
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,{7ABB600D-1E77-41BE-9FE5-B9142CFC08FA},2011-05-31 00:00:00


In [75]:
pd.merge(soh, sod, on = 'SalesOrderID').head(3)

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid_x,ModifiedDate_x,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid_y,ModifiedDate_y
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,{B207C96D-D9E6-402B-8470-2CC176C42283},2011-05-31 00:00:00
1,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,{7ABB600D-1E77-41BE-9FE5-B9142CFC08FA},2011-05-31 00:00:00
2,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00,3,4911-403C-98,1,778,1,2024.994,0.0,2024.994,{475CF8C6-49F6-486E-B0AD-AFC6A50CDD2F},2011-05-31 00:00:00


In [76]:
pd.merge(soh, sod, on = 'SalesOrderID')

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid_x,ModifiedDate_x,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid_y,ModifiedDate_y
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00,1,4911-403C-98,1,776,1,2024.9940,0.0,2024.9940,{B207C96D-D9E6-402B-8470-2CC176C42283},2011-05-31 00:00:00
1,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00,2,4911-403C-98,3,777,1,2024.9940,0.0,6074.9820,{7ABB600D-1E77-41BE-9FE5-B9142CFC08FA},2011-05-31 00:00:00
2,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00,3,4911-403C-98,1,778,1,2024.9940,0.0,2024.9940,{475CF8C6-49F6-486E-B0AD-AFC6A50CDD2F},2011-05-31 00:00:00
3,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00,4,4911-403C-98,1,771,1,2039.9940,0.0,2039.9940,{04C4DE91-5815-45D6-8670-F462719FBCE3},2011-05-31 00:00:00
4,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00,5,4911-403C-98,1,772,1,2039.9940,0.0,2039.9940,{5A74C7D2-E641-438E-A7AC-37BF23280301},2011-05-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,43892,8,2011-07-01 00:00:00,2011-07-13 00:00:00,2011-07-08 00:00:00,5,0,SO43892,PO2523117473,10-4020-000167,29716,281.0,4,1015,1015,5,13468.0,26813Vi69778,NaN,16544.5461,1587.1682,495.9901,18627.7044,NaN,{386511B7-D278-479C-90B9-687FF0E045BD},2011-07-08 00:00:00,996,0D3C-4E1F-9C,2,765,1,419.4589,0.0,838.9178,{30EDDEA5-2C93-4F08-B66E-BFADB84AFF0E},2011-07-01 00:00:00
996,43893,8,2011-07-01 00:00:00,2011-07-13 00:00:00,2011-07-08 00:00:00,5,0,SO43893,PO2204129382,10-4020-000370,29905,282.0,6,505,505,5,18480.0,116815Vi96000,407.0,419.4589,40.2681,12.5838,472.3108,NaN,{F3AFB011-5243-435C-9895-A520A4FC972D},2011-07-08 00:00:00,997,7C5D-46AA-A7,1,770,1,419.4589,0.0,419.4589,{46A1DA27-B11D-4918-8308-4B7715C12FB5},2011-07-01 00:00:00
997,43894,8,2011-07-01 00:00:00,2011-07-13 00:00:00,2011-07-08 00:00:00,5,0,SO43894,PO2958194987,10-4020-000559,30004,280.0,1,892,892,5,4842.0,46884Vi25430,NaN,32492.6040,3118.7048,974.5952,36585.9040,NaN,{BB847425-22C1-4432-BF39-88E9C7CEE13A},2011-07-08 00:00:00,998,7DF2-4E54-B0,3,710,1,5.7000,0.0,17.1000,{6005BBFE-CD7C-45FD-80F7-DBBADB10BFEC},2011-07-01 00:00:00
998,43894,8,2011-07-01 00:00:00,2011-07-13 00:00:00,2011-07-08 00:00:00,5,0,SO43894,PO2958194987,10-4020-000559,30004,280.0,1,892,892,5,4842.0,46884Vi25430,NaN,32492.6040,3118.7048,974.5952,36585.9040,NaN,{BB847425-22C1-4432-BF39-88E9C7CEE13A},2011-07-08 00:00:00,999,7DF2-4E54-B0,2,777,1,2024.9940,0.0,4049.9880,{03FF934E-043F-4CE7-8F44-18BFEE7EDBB7},2011-07-01 00:00:00


In [77]:
soh.columns

Index(['SalesOrderID', 'RevisionNumber', 'OrderDate', 'DueDate', 'ShipDate',
       'Status', 'OnlineOrderFlag', 'SalesOrderNumber', 'PurchaseOrderNumber',
       'AccountNumber', 'CustomerID', 'SalesPersonID', 'TerritoryID',
       'BillToAddressID', 'ShipToAddressID', 'ShipMethodID', 'CreditCardID',
       'CreditCardApprovalCode', 'CurrencyRateID', 'SubTotal', 'TaxAmt',
       'Freight', 'TotalDue', 'Comment', 'rowguid', 'ModifiedDate'],
      dtype='object')

In [78]:
sod.columns

Index(['SalesOrderID', 'SalesOrderDetailID', 'CarrierTrackingNumber',
       'OrderQty', 'ProductID', 'SpecialOfferID', 'UnitPrice',
       'UnitPriceDiscount', 'LineTotal', 'rowguid', 'ModifiedDate'],
      dtype='object')

In [79]:
soh.columns & sod.columns

<ipython-input-79-ea2082db8345>:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  soh.columns & sod.columns


Index(['SalesOrderID', 'rowguid', 'ModifiedDate'], dtype='object')

In [81]:
a = np.intersect1d(soh.columns, sod.columns)

print ("Common Columns:",a)

Common Columns: ['ModifiedDate' 'SalesOrderID' 'rowguid']


In [82]:
len(soh.columns)

26

In [83]:
len(sod.columns)

11

In [84]:
soh.columns.difference(sod.columns)

Index(['AccountNumber', 'BillToAddressID', 'Comment', 'CreditCardApprovalCode',
       'CreditCardID', 'CurrencyRateID', 'CustomerID', 'DueDate', 'Freight',
       'OnlineOrderFlag', 'OrderDate', 'PurchaseOrderNumber', 'RevisionNumber',
       'SalesOrderNumber', 'SalesPersonID', 'ShipDate', 'ShipMethodID',
       'ShipToAddressID', 'Status', 'SubTotal', 'TaxAmt', 'TerritoryID',
       'TotalDue'],
      dtype='object')

___
# Summary

You've seen how to use pandas DataFrames to clean your data and extract information. These are skills we'll continue to build in the course. 😀

### Check for Understanding

- What does a left join do?
- What's the pandas method for doing split-apply-combine operations?
- How do you stack two DataFrames on top of each other?
